<a href="https://colab.research.google.com/github/RemoteAge/Bracket/blob/master/Mentorcapaciteit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymysql

!pip install pandas

!pip install sshtunnel



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: paramiko
    Found existing installation: paramiko 4.0.0
    Uninstalling paramiko-4.0.0:
      Successfully uninstalled paramiko-4.0.0


Deze query haalt van een bepaalde maand van een jaar de laatste tijd van een controle (marked_at) op van een bepaalde mentor_id op

In [3]:
jaar = 2025
maand = 5
limiet = 10

qryLaatsteDag1 = ("""select
    ls.mentor_id AS id,
    count(0) AS aantal,
    max(ls.marked_at) AS marked_at_laatst,
    cast(ls.marked_at as date) AS marked_at_date
from learnnn.lesson_students ls
where
    ls.mentor_id is not null
    and ls.marked_at is not null
    and month(marked_at) = % s
    and year(marked_at) = % s
group by
    ls.mentor_id,
    cast(ls.marked_at as date)
order by
    marked_at
                    limit 10""")

Deze query haalt bij de mentor van de vorige query de eerste tijd van de volgende dag (bepaald in de vorige query) op

In [4]:
# de query die de laatste tijd waarop de mentor


qryEersteDag2 = ("""select
    ls.mentor_id AS id,
    count(0) AS aantal,
    min(ls.marked_at) AS marked_at_eerst,
    cast(ls.marked_at as date) AS market_at_date
from learnnn.lesson_students ls
where
    ls.mentor_id is not null
    and ls.marked_at is not null
    and mentor_id = %s and
    cast(ls.marked_at as date) = %s
group by
    ls.mentor_id,
    cast(ls.marked_at as date)
    """)




Deze query bepaald hoeveel studenten moeten nagekeken worden maw het aantal studenten met een finished_at tussen de max marked_at tijd van dag 1 (eerste query) en de minimale marked at tijd op dag 2  

In [5]:

qryAantalWachtend =  ("""select
    count(*) as aantal
 from learnnn.lesson_students
where finished_at BETWEEN %s AND %s
AND mentor_id = %s""")

Het enige wat we dan nog moeten bepalen is; hoeveel na te kijken studenten zijn er nog over als de mentor stopt met nakijken. Dit is echter niet zo makkelijk te bepalen want hoe weet ik wanneer een mentor gestopt is???

In [23]:
#query met niet nagegekeken studenten
#dit is dezelfde query ongeveer als hierboven

qryAantalgedaanIn2uur = ("""select
    count(*) as aantal
 from learnnn.lesson_students
where marked_at BETWEEN %s AND %s
AND mentor_id = %s""")

#Uitvoerende code

Hier onder worden de queries uitgevoerd



In [24]:
import pymysql
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
from datetime import datetime, timedelta

home = expanduser('~')

# if you want to use ssh password use - ssh_password='your ssh password', bellow

sql_hostname = 'learnnn.cq5da2uzz5pr.eu-west-1.rds.amazonaws.com'
sql_username = 'dbreaduser'
sql_password = '728PNY4lvHQeDz%A69i3gF20'
sql_main_database = 'learnnn'
sql_port = 3306
ssh_host = 'learnnn.com'
ssh_user = 'dbreaduser'
ssh_port = 22
twee_uur = timedelta(hours=2)

with SSHTunnelForwarder(
       (ssh_host, ssh_port),
       ssh_username=ssh_user,
       ssh_password=sql_password,
       remote_bind_address=(sql_hostname, sql_port)) as tunnel:
       conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                          passwd=sql_password, db=sql_main_database,
                          port=tunnel.local_bind_port)



       with conn.cursor() as cursor:
        cursor.execute(qryLaatsteDag1,(5,2025) )
        columns = [column[0] for column in cursor.description]

        result = cursor.fetchall()
        for row in result:
            mentorId = row[0]
            tijdDag1 = row[2]
            datum = row[3]
            next_day = datum + timedelta(days=1)
            test = f"test {mentorId} {datum} {row[2]}"
            #print(test) #  "mentor: " + mentorId )
            with conn.cursor() as cursor2:
                cursor2.execute(qryEersteDag2,(mentorId,next_day))
                result2 = cursor2.fetchall()
                for row2 in result2:
                   tijdDag2 = row2[2]
                   #print(f"{mentorId} {tijdDag1} {tijdDag2} ")
                   with conn.cursor() as cursor3:
                        cursor3.execute(qryAantalWachtend,(tijdDag1,tijdDag2,mentorId) )
                        result3 = cursor3.fetchall()
                        with conn.cursor() as cursor4:

                            cursor4.execute(qryAantalgedaanIn2uur,(tijdDag2,tijdDag2 + twee_uur,mentorId) )
                            result4 = cursor4.fetchall()
                        #print(f"{mentorId} {tijdDag1} {tijdDag2} {result3[0][0]}")
                        print(f"{mentorId} {tijdDag1} {tijdDag2} {result3[0][0]} {result4[0][0]}")
        conn.close()

1097755 2025-05-01 16:04:41 2025-05-02 01:23:05 10 2
1112558 2025-05-01 22:47:06 2025-05-02 20:22:17 8 8
668109 2025-05-01 00:28:53 2025-05-02 03:32:03 7 3
1027990 2025-05-01 21:36:08 2025-05-02 19:19:24 18 18
452758 2025-05-01 19:08:57 2025-05-02 17:00:53 1 2
800557 2025-05-01 01:40:19 2025-05-02 09:11:33 51 35
1215264 2025-05-01 09:00:49 2025-05-02 01:37:25 2 2
